In [4]:
#Created by Indrajit Ghosh

#A POC based on Movie Recommender System using Pinecone Vector DB
#The code will be connecting to the pinecone cloud system via api key, for the 
#dataset , I have used an IMDB dataset having 1000 movie records
#The overview field of the dataset is embedded into vector 
#The embedded vector is uploaded to the pinecone DB
#Query semantic search is based on the required movie type by description
#The relevant movie names are fetched by the vector db 

In [5]:
import pinecone
print(pinecone.__version__)
import pandas as pd


5.0.1


In [32]:
import pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone

# Check Pinecone version
print(pinecone.__version__)

# Initialize Pinecone with API key
pc = Pinecone(api_key="apikey")
# Create index using ServerlessSpec
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]
index_name="recomendar"
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,  
        metric="cosine", 
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

print("Index created successfully!")


5.0.1
Index created successfully!


In [7]:
movie_1000_dataset=pd.read_csv("imdb_top_1000.csv")
refined_dataset=movie_1000_dataset[['Series_Title','Overview','Genre','IMDB_Rating']].dropna()

In [18]:
refined_dataset.head()


,Series_Title,Overview,Genre,IMDB_Rating,movie_id,embedding
0,The Shawshank Redemption,"Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",Drama,9.3,movie-0,"[-0.06326335668563843, 0.0414624847471714, -0.04707525670528412, -0.03361903131008148, -0.025629350915551186, 0.034998346120119095, 0.00804077833890915, -0.050420038402080536, 0.0021566669456660748, -0.03816813603043556, 0.060007140040397644, 0.05826715752482414, -0.011546925641596317, 0.045500513166189194, 0.014620830304920673, -0.031161485239863396, -0.116477370262146, 0.01240277849137783, 0.000569032272323966, 0.06585683673620224, 0.058869749307632446, -0.021372728049755096, -0.003250517649576068, -0.07673660665750504, 0.02135494537651539, 0.081763356924057, -0.00918985903263092, -0.06858592480421066, 0.04636621102690697, 0.0422762967646122, 0.07642403990030289, 0.01504435669630766, 0.12988883256912231, 0.04492688551545143, 0.03841068968176842, 0.02938127890229225, -0.0396609790623188, -0.004459293093532324, 0.026865225285291672, -0.08762043714523315, -0.021634984761476517, 0.012612489052116871, 0.04844161495566368, -0.0047754994593560696, 0.02080054208636284, -0.023759111762046814, -0.06199203431606293, -0.0021743830293416977, -0.029131552204489708, -0.028240825980901718, 0.003856562776491046, 0.022386766970157623, -0.06321794539690018, 0.05625985935330391, -0.0485297366976738, 0.016478506848216057, -0.005382803734391928, 0.046138182282447815, -0.07234018296003342, -0.029620008543133736, 0.019837070256471634, 0.04205045849084854, -0.0019076867029070854, -0.004228489939123392, -0.031020512804389, 0.008290155790746212, 0.047929547727108, 0.03191802650690079, 0.008747188374400139, 0.02626904472708702, 0.013517152518033981, -0.009413911029696465, -0.06911010295152664, -0.124101422727108, -0.013474998995661736, 0.020222151651978493, -0.08685393631458282, 0.027753980830311775, -0.0848717913031578, -0.08736436069011688, -0.007402233779430389, -0.04772278293967247, -0.04302262142300606, 0.01652613840997219, -0.01965237408876419, -0.04892267659306526, -0.04450356215238571, -0.055566176772117615, 0.04919380694627762, 0.019433923065662384, 0.020866597071290016, 0.0008050915785133839, 0.0003361470880918205, -0.01610475592315197, -0.09029942750930786, 0.02381822094321251, 0.00675453245639801, 0.11394044011831284, -0.07233685255050659, 0.07673592120409012, ...]"
1,The Godfather,An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.,"Crime, Drama",9.2,movie-1,"[-0.03172875568270683, 0.014316813088953495, -0.0615805983543396, -0.002287553157657385, 0.0021945720072835684, 0.031465038657188416, 0.03637707233428955, -0.01673746481537819, -0.06584277004003525, 0.00865146704018116, 0.0998104140162468, -0.004719762597233057, 0.06793434917926788, -0.05568943917751312, -0.022136930376291275, -0.05818478763103485, -0.03279603272676468, -0.013259630650281906, -0.04612188786268234, -0.015468133613467216, -0.05622775852680206, -0.11165951937437057, 0.11388993263244629, -0.06925135850906372, -0.0026519796811044216, 0.06188793107867241, -0.014782654121518135, -0.007336127571761608, 0.04265176132321358, -0.019056882709264755, -0.0039016283117234707, -0.031043050810694695, 0.014027034863829613, 0.04549039900302887, -0.005722604691982269, -0.058653730899095535, 0.06926244497299194, 0.026127301156520844, 0.010088388808071613, 0.013631844893097878, 0.09388367086648941, 0.023086844012141228, 0.027327610179781914, 0.0085598174482584, -0.038511745631694794, -0.08164053410291672, -0.05300317704677582, 0.022333407774567604, 0.0018544199410825968, 0.018248867243528366, 0.007823186926543713, -0.01847103051841259, 0.0402287021279335, 0.05249093472957611, -0.029454659670591354, -0.006331408862024546, 0.09823063015937805, -0.0460694245994091, 0.03143187612295151, -0.026294853538274765, -0.0005892703775316477, 0.07098889350891113, -0.024598868563771248, -0.06171102449297905, -0.027791500091

In [9]:
refined_dataset['movie_id'] = [f"movie-{i}" for i in range(len(refined_dataset))]



In [10]:
#Generation of Embeddings

from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for movie descriptions
refined_dataset['embedding'] = refined_dataset['Overview'].apply(lambda x: model.encode(x).tolist())

In [11]:
refined_dataset.head()

,Series_Title,Overview,Genre,IMDB_Rating,movie_id,embedding
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,9.3,movie-0,"[-0.06326335668563843, 0.0414624847471714, -0...."
1,The Godfather,An organized crime dynasty's aging patriarch t...,"Crime, Drama",9.2,movie-1,"[-0.03172875568270683, 0.014316813088953495, -..."
2,The Dark Knight,When the menace known as the Joker wreaks havo...,"Action, Crime, Drama",9.0,movie-2,"[0.018887894228100777, 0.05393049120903015, -0..."
3,The Godfather: Part II,The early life and career of Vito Corleone in ...,"Crime, Drama",9.0,movie-3,"[-0.05933509021997452, 0.05894988402724266, -0..."
4,12 Angry Men,A jury holdout attempts to prevent a miscarria...,"Crime, Drama",9.0,movie-4,"[-0.03015812113881111, 0.0782327651977539, 0.0..."


In [12]:
#No Need to run multiple time , one time activity of embedding
# Connect to the index
index = pc.Index(index_name)

# Prepare data for upload
to_upsert = [
    (
        row['movie_id'],  # Unique identifier for Pinecone
        row['embedding'],  # Embedding vector
        {  # Metadata dictionary
            'title': row['Series_Title'],  
            'overview': row['Overview'], 
            'genre': row['Genre'], 
            'rating': row['IMDB_Rating']  
        }
    )
    for _, row in refined_dataset.iterrows()
]


# Upsert data into Pinecone DB
index.upsert(vectors=to_upsert)


{'upserted_count': 1000}

In [28]:
# Define a query (e.g., a description of a movie you like)
query = "show me a murder mystery"

# Convert the query to an embedding
query_embedding = model.encode(query).tolist()

# Query Pinecone for similar movies
# Perform the query
results = index.query(
    vector=query_embedding,
    top_k=3,
    include_values=False,
    include_metadata=True
)


#print(results)

In [29]:
report=[]
for match in results['matches']:
    title=match['metadata']['title']
    genre=match['metadata']['genre']
    rating=match['metadata']['rating']
    overview=match['metadata']['overview']
    report.append({'Title':title,'Genre':genre,'Overview':overview,'Rating':rating})





In [30]:
pd.set_option('display.max_colwidth', None)  
df = pd.DataFrame(report)
df.head()

,Title,Genre,Overview,Rating
0,Knives Out,"Comedy, Crime, Drama","A detective investigates the death of a patriarch of an eccentric, combative family.",7.9
1,The Thin Man,"Comedy, Crime, Mystery","Former detective Nick Charles and his wealthy wife Nora investigate a murder case, mostly for the fun of it.",8.0
2,Madeo,"Crime, Drama, Mystery",A mother desperately searches for the killer who framed her son for a girl's horrific murder.,7.8


In [16]:
#Check the vector db stats
index_stats = index.describe_index_stats()

In [17]:
print(index_stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}
